In [16]:
import json
import subprocess
import os

In [17]:
with open('../1-coding/diffs.json', 'r') as f:
    diffs = json.load(f)

In [18]:
CHECKOV_CHECKS = ','.join({'CKV_AWS_801', 'CKV_AWS_802', 'CKV_AWS_803'})

In [19]:
stats = []

for d in diffs:
    _, _, _, owner, name, _, sha = d['url'].split('/')

    snapshot_path = f'snapshots/{owner}-{name}-{sha}'

    summary = {
        'url': d['url'],
        'before': [],
    }

    for version in os.listdir(snapshot_path):
        version_path = f'{snapshot_path}/{version}/'

        checkov_result = subprocess.run([
            'checkov',
            '--evaluate-variables', 'True',
            '-d', version_path,
            '-c', CHECKOV_CHECKS,
        ], capture_output=True)

        print(checkov_result)

        if version == 'after':
            summary['after'] = {
                'return_code': checkov_result.returncode,
                'stdout': checkov_result.stdout,
                'stderr': checkov_result.stderr,
            }
        else:
            summary['before'].append({
                'return_code': checkov_result.returncode,
                'stdout': checkov_result.stdout,
                'stderr': checkov_result.stderr,
            })

        stats.append(summary)

    break

CompletedProcess(args=['checkov', '--evaluate-variables', 'True', '-d', 'snapshots/tkhoa2711-terraform-digitalocean-a86d89369aaf5a20c1e4d8415a8a771aa7de7d10/after/', '-c', 'CKV_AWS_803,CKV_AWS_801,CKV_AWS_802'], returncode=0, stdout=b"\n       _               _              \n   ___| |__   ___  ___| | _______   __\n  / __| '_ \\ / _ \\/ __| |/ / _ \\ \\ / /\n | (__| | | |  __/ (__|   < (_) \\ V / \n  \\___|_| |_|\\___|\\___|_|\\_\\___/ \\_/  \n                                      \nBy Prisma Cloud | version: 3.2.999 \n\n\n", stderr=b'')
CompletedProcess(args=['checkov', '--evaluate-variables', 'True', '-d', 'snapshots/tkhoa2711-terraform-digitalocean-a86d89369aaf5a20c1e4d8415a8a771aa7de7d10/before-625a17182bd5b1770613a0606df75ebe7510af92/', '-c', 'CKV_AWS_803,CKV_AWS_801,CKV_AWS_802'], returncode=0, stdout=b"\n       _               _              \n   ___| |__   ___  ___| | _______   __\n  / __| '_ \\ / _ \\/ __| |/ / _ \\ \\ / /\n | (__| | | |  __/ (__|   < (_) \\ V / \n  \\___|_| |

In [20]:
import polars as pl

df = pl.DataFrame(stats)

print(df)

shape: (2, 3)
┌────────────────────────────────┬────────────────────────────────┬────────────────────────────────┐
│ url                            ┆ before                         ┆ after                          │
│ ---                            ┆ ---                            ┆ ---                            │
│ str                            ┆ list[struct[3]]                ┆ struct[3]                      │
╞════════════════════════════════╪════════════════════════════════╪════════════════════════════════╡
│ https://github.com/tkhoa2711/t ┆ [{0,b"\x0a\x20\x20\x20\x20\x20 ┆ {0,b"\x0a\x20\x20\x20\x20\x20\ │
│ …                              ┆ …                              ┆ …                              │
│ https://github.com/tkhoa2711/t ┆ [{0,b"\x0a\x20\x20\x20\x20\x20 ┆ {0,b"\x0a\x20\x20\x20\x20\x20\ │
│ …                              ┆ …                              ┆ …                              │
└────────────────────────────────┴────────────────────────────────┴──────────